In [2]:
import pyaudio
import numpy as np
import time

In [9]:
"""
基本参数
"""
sr = 44100
chunk = 1024
num_channel = 1
record_second = 1
chunks_per_buffer = int(sr/chunk)



"""
关心的频率范围
"""
bandsize = sr/chunk

bass_indexL = int(np.floor(60/bandsize))
bass_indexR = int(np.ceil(130/bandsize))

midrange_indexL = int(np.floor(301/bandsize))
midrange_indexR = int(np.ceil(750/bandsize))

"""
实时读取 stream 数据并分析
"""

p = pyaudio.PyAudio() 
stream = p.open(format=pyaudio.paInt16, channels=1, rate=sr, input=True,
                frames_per_buffer=chunk)

In [15]:
"""
以 1 秒钟为单位 track beat
sr = 44100
chunk = 1024
1 秒钟约 43 chunk
buffer 里需要存 43 个振幅值
两个频率范围需要两个 buffer 存储数据
因是实时分析，每读取分析完一个 chunk 并与 buffer 均值比较判断完 beat 后，要提换掉 buffer 内最老的那个 chunk 值
"""
buffer_bass = []
buffer_midrange = []  

for i in np.arange(chunks_per_buffer):
    block = stream.read(int(chunk))
    data = np.fromstring(block, dtype=np.int16)
    data_norm = data/(2**15)
    y = np.abs(np.fft.fft(data_norm))[0:int(chunk/2)]/chunk
    y[1:] = 2*y[1:]
    y_bass = np.mean(y[bass_indexL:bass_indexR])
    y_midrange = np.mean(y[midrange_indexL:midrange_indexR])
    buffer_bass.append(y_bass)
    buffer_midrange.append(y_midrange)

bhistory_bass = np.mean(buffer_bass)
bhistory_midrange = np.mean(buffer_midrange)
                        
"""
计算两个频率范围对应 buffer 的方差与判断 beat 的阈值
"""
    
var_buffer_bass = np.mean([np.power((i-bhistory_bass)/np.max(buffer_bass),2) 
                           for i in buffer_bass])

var_buffer_midrange = np.mean([np.power((i-bhistory_midrange)/np.max(buffer_midrange),2) for i in buffer_midrange])

threshold_bass = - 15*var_buffer_bass + 1.55

threshold_midrange = -15*var_buffer_midrange + 1.55
                        
"""
上一栏算是先生成第 1 秒 buffer 以及 beat track 阈值，从下面开始就可以 beat track 了

"""

print("go!")
while True:
    block = stream.read(int(chunk))
    data = np.fromstring(block, dtype=np.int16)
    data_norm = data/(2**15)
    y = np.abs(np.fft.fft(data_norm))[0:int(chunk/2)]/chunk
    y[1:] = 2*y[1:]
    y_bass = np.mean(y[bass_indexL:bass_indexR])
    if y_bass > threshold_bass:
                        print ("bass beat!")
    y_midrange = np.mean(y[midrange_indexL:midrange_indexR])
    if y_midrange > threshold_midrange:
                        print("midrange beat!")
    buffer_bass.append(y_bass)
    buffer_midrange.append(y_midrange)
    
    buffer_bass = buffer_bass[1:]
    buffer_midrange = buffer_midrange[1:]
                        
    bhistory_bass = np.mean(buffer_bass)
    bhistory_midrange = np.mean(buffer_midrange)
                        
    var_buffer_bass = np.mean([np.power((i-bhistory_bass)/np.max(buffer_bass),2) for i in buffer_bass])
    var_buffer_midrange = np.mean([np.power((i-bhistory_midrange)/np.max(buffer_midrange),2) for i in buffer_midrange])

    threshold_bass = - 15*var_buffer_bass + 1.55
    threshold_midrange = -15*var_buffer_midrange + 1.55

KeyboardInterrupt: 

In [16]:
y_bass

0.0

In [17]:
y_midrange

0.0

In [18]:
threshold_bass

nan